# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [1]:
import pandapower as pp
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3)
pp.create_load(net, bus3, p_kw=70e3)
pp.create_load(net, bus4, p_kw=10e3)

#create generators
eg = pp.create_ext_grid(net, bus1)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=0, max_p_kw=-80e3,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=0, max_p_kw=-100e3, vm_pu=1.01, controllable=True)

## Loss Minimization

We run an OPF:

In [2]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [3]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.143432,-1974.468319


In [4]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.741033,1969.652645,-3.712797,1.000009
1,-12299.405301,1451.157657,-3.712788,1.000010


Since no individiual generation costs were specified, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [5]:
net.ext_grid.loc[eg, "cost_per_kw"] = 0.10
net.gen.loc[g0, "cost_per_kw"] = 0.15
net.gen.loc[g1, "cost_per_kw"] = 0.12

And now run an OPF:

In [6]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

We define a simple function that calculates the summed costs:

In [7]:
def calc_costs(net):
    cost_gen = (-net.res_gen.p_kw * net.gen.cost_per_kw).sum()
    cost_eg = (-net.res_ext_grid.p_kw * net.ext_grid.cost_per_kw).sum()
    return (cost_gen + cost_eg) * 1e-3

And calculate the dispatch costs:

In [8]:
calc_costs(net)

14.455950398593108

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [9]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,144559.441044,9193.041306,-143959.512544,15993.765654,599.9285,25186.80696,0.380136,0.759988,144.851455


We now limit the transformer loading to 50%:

In [10]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [11]:
pp.runopp(net)

We can see that the transformer complies with the maximum loading:

In [12]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49997.692608,-474.065938,-49877.753652,3494.099713,119.938955,3020.033775,0.131216,0.262148,49.99994


And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [13]:
net.res_ext_grid

,p_kw,q_kvar
0,-49997.692656,474.065939


In [14]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-0.012190,-2453.354893,-6.222616,0.981741
1,-93274.501055,-2381.606656,-1.195302,1.022312


This comes of course with an increase in dispatch costs:

In [15]:
calc_costs(net)

16.192711220700627

### Line Loading Constraints

Wen now look at the line loadings:

In [16]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,19816.827917,-1889.701046,-19376.454790,531.676027,440.373128,-1358.025019,0.104370,0.103630,0.104370,22.206433
1,-50623.532796,1921.679076,52761.258328,925.681591,2137.725532,2847.360667,0.270841,0.270923,0.270923,57.643101
2,30513.242385,1455.925693,-29939.073952,-1604.398770,574.168433,-148.473077,0.156836,0.157195,0.157195,33.445685


and run the OPF with a 50% loading constraint:

In [17]:
net.line["max_loading_percent"] = 50
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Now the line loading constraint is complied with:

In [18]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,16302.437096,-1966.957654,-16004.087049,300.190085,298.350047,-1666.767569,0.086093,0.085251,0.086093,18.317675
1,-43226.657575,1353.756962,44772.860492,241.636788,1546.202917,1595.393749,0.230334,0.230441,0.230441,49.030086
2,26871.931708,1113.113163,-26424.615789,-1526.238188,447.315919,-413.125025,0.138424,0.138774,0.138774,29.526460


And all generators are involved in supplying the loads:

In [19]:
net.res_ext_grid

,p_kw,q_kvar
0,-49997.760348,473.155031


In [20]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-10769.255371,-1653.947042,-5.732452,0.985494
1,-81644.792208,-1354.749936,-1.454455,1.019782


This of course comes with a once again rising dispatch cost:

In [21]:
calc_costs(net)

16.412539405473442

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [22]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49997.760348,473.155031
1,1.001081,-3.433990,60000.000000,0.000000
2,0.985494,-5.732452,59230.744629,-1653.947042
3,1.019782,-1.454455,-71644.792208,-1354.749936


and constrain it:

In [23]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net)

We can see that all voltages are within the voltage band:

In [24]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49752.609473,3045.859455
1,1.004155,-3.410474,60000.000000,0.000000
2,1.000000,-5.896052,58075.533600,-14192.697949
3,1.020000,-1.382763,-70725.716975,8717.600450


And all generators are once again involved in supplying the loads:

In [25]:
net.res_ext_grid

,p_kw,q_kvar
0,-49752.609473,3045.859455


In [26]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-11924.466400,-14192.697949,-5.896052,1.00
1,-80725.716975,8717.600450,-1.382763,1.02


This of course comes once again with rising dispatch costs:

In [27]:
calc_costs(net)

16.451016944344822

## Minimizing Active Power Curtailment

Now we assume that the generators are renewable energy sources, and we want to feed in as much of the energy they can provide as possible without violating any constraints.

We assign negative costs to the generators and costs of zero for the external grid.

In [28]:
net.ext_grid.cost_per_kw = 0
net.gen.cost_per_kw = -.1
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Because of the negative costs, the OPF now maximizes power generation at the generators:

In [29]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-48973.329354,-17397.619927,-0.322158,1.02
1,-82154.593187,18290.265685,2.800892,1.02


In [30]:
net.res_ext_grid

,p_kw,q_kvar
0,-10941.402293,152.853706


In [31]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.00,0.000000,-10941.402293,152.853706
1,1.00,-0.750481,60000.000000,0.000000
2,1.02,-0.322158,21026.670646,-17397.619927
3,1.02,2.800892,-72154.593187,18290.265685


In [32]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,10941.402293,-152.853706,-10883.307761,319.769684,58.094532,166.915978,0.028717,0.057147,10.94247


In [33]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-5881.959775,-6801.024106,5956.449698,4583.106489,74.489923,-2217.917616,0.047194,0.038673,0.047194,10.041360
1,-26983.120344,12814.513438,27688.320273,-13054.411518,705.199929,-239.898080,0.153710,0.157518,0.157518,33.514483
2,44466.272914,-5235.854166,-43234.732464,6481.254422,1231.540450,1245.400256,0.230392,0.229459,0.230392,49.019560


Obviously the voltage profile was the limiting factor for the generator feed-in. If we relax this constraint a little bit:

In [34]:
net.bus.max_vm_pu = 1.05
pp.runopp(net)

We see an increased feed-in of the generators:

In [35]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-79999.993259,-22578.054433,2.046851,1.050000
1,-64283.607310,17309.567754,3.523663,1.032947


In [36]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,2318.942554,6806.821245
1,1.008245,0.149907,60000.000000,0.000000
2,1.050000,2.046851,-9999.993259,-22578.054433
3,1.032947,3.523663,-54283.607310,17309.567754
